# Rust & Excel test item list matching


In [105]:
FILE_TEST_LITEMS = "./data/test_items.xlsx"
PATH_RUST = "./rust/test/*.rs"
PATH_CSV = "./out.csv"


## 1. Make list of Rust test functions

In [106]:
# Make list of Rust test function
import os
import csv
import glob

header = ["No.", "file", "test_func", "feature1", "feature2", "feature3"]
feature_list = []
test_lines = []

def extract_test_fnc(rust_file_path, count):

    basename_rust = os.path.basename(rust_file_path)

    with open(rust_file_path) as f:
        lines = f.readlines()
        with open(PATH_CSV, "a") as fc:
            writer = csv.writer(fc, quoting=csv.QUOTE_NONE)
            for i, line in enumerate(lines):
                if ("#[test]" in line):
                    test_lines.append(i)
                elif ("#[cfg(feature =" in line):
                    feature_name = line.strip().replace("#[cfg(feature = \"", "").replace("\")]", "")
                    feature_list.append(feature_name)
                elif ("pub fn" in line):
                    feature_list.clear()
                elif ("fn " in line) and ( (i-1 in test_lines) or (i-2 in test_lines) ):
                    func_name = line.strip().replace("fn ", "").replace("()", "").replace(" {", "")
                    feature_cnt = len(feature_list)
                    count = count + 1
                    if (feature_cnt == 0):
                        writer.writerow([count, basename_rust, func_name, "", "", ""])
                    elif (feature_cnt == 1):
                        writer.writerow([count, basename_rust, func_name, feature_list[0], "", ""])
                    elif (feature_cnt == 2):
                        writer.writerow([count, basename_rust, func_name, feature_list[0], feature_list[1], ""])
                    elif (feature_cnt == 3):
                        writer.writerow([count, basename_rust, func_name, feature_list[0], feature_list[1], feature_list[2]])
                    else:
                        print("Unsupported Format!! ", basename_rust, line)
                    feature_list.clear()
    return count

def main():
    print("List up rust test function in[", PATH_RUST, "]")

    cnt = 0

    if os.path.exists(PATH_CSV):
        os.remove(PATH_CSV)

    with open(PATH_CSV, "a") as fc:
        writer = csv.writer(fc, quoting=csv.QUOTE_NONE)
        writer.writerow(header)
    files = glob.glob(PATH_RUST)
    for file in files:
        ret = extract_test_fnc(file, cnt)
        cnt = cnt + ret
    print("Completed...")
    print("CSV file[", PATH_CSV, "].")
    print("Total ", cnt ," functions are listed.")

if __name__ == "__main__":
    main()


List up rust test function in[ ./rust/test/*.rs ]
Completed...
CSV file[ ./out.csv ].
Total  9  functions are listed.


## 2. Select data using Pands

In [107]:
import pandas as pd
import openpyxl
import glob

### 2-1. Check CSV file (Rust test function list)

In [108]:
# show table in jupyterNotebook
test_funcs_csv = pd.read_csv(PATH_CSV)
test_funcs_csv

,No.,file,test_func,feature1,feature2,feature3
0,1,test1.rs,test_parallel1,parallel,serde,NaN
1,2,test1.rs,test_serde1,serde,NaN,NaN
2,3,test1.rs,test_special1,special,NaN,NaN
3,4,test2.rs,test_parallel2,parallel,serde,special
4,5,test2.rs,test_serde2,serde,NaN,NaN
5,6,test2.rs,test_special2,special,NaN,NaN


### 2-2. Check Excel file(Test item list)
* After creation by free app "NUMBERS", Excel file includes a sheet names "書き出しの概要".
* We should exclude this first sheet later.

In [109]:
in_file = pd.ExcelFile(FILE_TEST_LITEMS)
in_file.sheet_names

['書き出しの概要', 'Func1', 'Func2']

### 2-3. Pick up data from Excel
Choose following items of Excel data format.

* Sheet name
* Column "No."
* Column "テスト関数"

In [110]:
test_items = []

for sheet_name in in_file.sheet_names:
    # exclude sheet made by numbers app.
    if sheet_name != '書き出しの概要':
        sheet = pd.read_excel("./data/test_items.xlsx", sheet_name)
        # print(sheet)
        for index, raw in sheet.iterrows():
            data = []
            data.append(sheet_name)
            data.append(raw["No."])
            data.append(raw["テスト関数"])
            # print(data)
            test_items.append(data)

print(test_items)

[['Func1', 1, 'test_parallel1'], ['Func1', 2, 'test_serde1'], ['Func1', 3, nan], ['Func1', 4, 'test_serde1'], ['Func2', 1, 'test_parallel2'], ['Func2', 2, 'test_serde2'], ['Func2', 3, 'test_special2'], ['Func2', 4, 'test_another2']]


In [111]:
# import to pandas DataFrame
test_items_excel = pd.DataFrame(test_items,
                  columns=['sheet_name', 'test_number', 'test_func'])
test_items_excel


,sheet_name,test_number,test_func
0,Func1,1,test_parallel1
1,Func1,2,test_serde1
2,Func1,3,NaN
3,Func1,4,test_serde1
4,Func2,1,test_parallel2
5,Func2,2,test_serde2
6,Func2,3,test_special2
7,Func2,4,test_another2


# 3. Check results

## 3-1. Rust test_func check

* check `sheet_name` is "NaN" valid.

In [112]:
# print(test_funcs_csv)

merged_left_rust = pd.merge(test_funcs_csv, test_items_excel, how='left', on='test_func')
merged_left_rust


,No.,file,test_func,feature1,feature2,feature3,sheet_name,test_number
0,1,test1.rs,test_parallel1,parallel,serde,NaN,Func1,1.0
1,2,test1.rs,test_serde1,serde,NaN,NaN,Func1,2.0
2,2,test1.rs,test_serde1,serde,NaN,NaN,Func1,4.0
3,3,test1.rs,test_special1,special,NaN,NaN,NaN,NaN
4,4,test2.rs,test_parallel2,parallel,serde,special,Func2,1.0
5,5,test2.rs,test_serde2,serde,NaN,NaN,Func2,2.0
6,6,test2.rs,test_special2,special,NaN,NaN,Func2,3.0


## 3-2. Check lost function(Not implemented, Deleted)

* check `test_func` is "NaN" valid.

In [113]:
merged_left_items = pd.merge(test_items_excel, test_funcs_csv, how='left', on='test_func')
merged_left_items

,sheet_name,test_number,test_func,No.,file,feature1,feature2,feature3
0,Func1,1,test_parallel1,1.0,test1.rs,parallel,serde,NaN
1,Func1,2,test_serde1,2.0,test1.rs,serde,NaN,NaN
2,Func1,3,NaN,NaN,NaN,NaN,NaN,NaN
3,Func1,4,test_serde1,2.0,test1.rs,serde,NaN,NaN
4,Func2,1,test_parallel2,4.0,test2.rs,parallel,serde,special
5,Func2,2,test_serde2,5.0,test2.rs,serde,NaN,NaN
6,Func2,3,test_special2,6.0,test2.rs,special,NaN,NaN
7,Func2,4,test_another2,NaN,NaN,NaN,NaN,NaN


# 4. Export to Excel file

In [114]:
merged_left_rust.to_excel("./01_check_not_listed_funcs.xlsx", sheet_name='result')

merged_left_items.to_excel("./02_check_not_implemented_funcs.xlsx", sheet_name='result')
